In [79]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from transformers import  BertTokenizer, TFAlbertModel
import implicit
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import tensorflow as tf


In [80]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
numpyArray = np.load('../Jiro/features.npy')
np.load = np_load_old
ratings_ = df = pd.DataFrame(numpyArray, columns = ['user_id','business_id','stars','text'])

In [81]:
ratingsnew = ratings_.head(50000)
ratingsnew["stars"].astype(float).round()

for count in range(len(ratingsnew.text)):
    ratingsnew.at[count, 'text'] = " ".join(ratingsnew.at[count, 'text'])

ratingsnew['text'] = ratingsnew['text'].astype(str)
print(ratings_.head(1000))


                    user_id             business_id stars  \
0    mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   3.0   
1    OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   5.0   
2    8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   3.0   
3    _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   5.0   
4    bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   4.0   
..                      ...                     ...   ...   
995  syyKcKPFILDysHWmtka-aA  1_hDCN3iioFR3XnUr32ZtA   5.0   
996  _lgLNzpzf3qmbwySBakxEw  5RzJ2bjU8bLSaN5SuiUpYA   4.0   
997  Sh_vUlHHY2Kuj14eF8NYZQ  s1PNBO9o5jIgNd5YWUDLXQ   5.0   
998  YwMD-AVT67fmYRGxnlRSPA  alUk6OwNhofyc90NDMDY-Q   5.0   
999  WKe2b_EeLBnZ3lZV5WKYGQ  -Or44IdY51Ukd618kikmtA   4.0   

                                                  text  
0    [CLS] If you decide to eat here , just be awar...  
1    [CLS] I ' ve taken a lot of spin classes over ...  
2    [CLS] Family diner . Had the b ##uff ##et . E ...  
3    [CLS] Wow ! Yu ##mmy , different ,

In [82]:
# df_features = df_ratings.pivot_table(
#     index = "user_id",
#     columns = "business_id",
#     values = "labels",
# ).fillna(0)
df_features = ratingsnew
df_features["user_id"]  = df_features['user_id'].astype("category").cat.codes
df_features["business_id"]  = df_features['business_id'].astype("category").cat.codes

In [83]:

sparse_item_user = sparse.csr_matrix((df_features['stars'].astype('float'), (df_features['user_id'], df_features['business_id'])))
sparse_user_item = sparse.csr_matrix((df_features['stars'].astype('float'),  (df_features['business_id'],df_features['user_id'])))

In [84]:
modelRec = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)


alpha_val = 40
data_conf = (sparse_item_user * alpha_val).astype('double')


modelRec.fit(data_conf)

  0%|          | 0/20 [00:00<?, ?it/s]

In [85]:
df_features["labels"] = df_features.apply(lambda row: modelRec.user_factors[row['user_id']],axis=1)
df_features.head()

,user_id,business_id,stars,text,labels
0,33928,4314,3.0,"[CLS] If you decide to eat here , just be awar...","[-0.21822962, -0.60841936, 0.43131685, -1.4994..."
1,17345,1063,5.0,[CLS] I ' ve taken a lot of spin classes over ...,"[1.9655457, 0.18740198, -0.20779221, -0.167741..."
2,6433,4485,3.0,[CLS] Family diner . Had the b ##uff ##et . E ...,"[0.89440405, -1.4331872, 0.91099584, 0.5890252..."
3,24893,6143,5.0,"[CLS] Wow ! Yu ##mmy , different , delicious ....","[1.4070368, 0.380417, 0.54042625, -0.44361296,..."
4,26602,5291,4.0,[CLS] Cut ##e interior and owner ( ? ) gave us...,"[-1.1844745, 1.3675942, -1.174313, 1.2371562, ..."


In [86]:
train_df, test_df = train_test_split(df_features, test_size = 0.2)
y_train = tf.convert_to_tensor(
    train_df["labels"].tolist(), dtype=float
)

y_test = tf.convert_to_tensor(
    test_df["labels"].tolist(), dtype=float
)

In [87]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
albert = TFAlbertModel.from_pretrained('roberta-base')

You are using a model of type roberta to instantiate a model of type albert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at roberta-base were not used when initializing TFAlbertModel: ['lm_head', 'roberta']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFAlbertModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['albert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
x_train = tokenizer(
    text=train_df.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=test_df.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [89]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [90]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [91]:
max_len = 70
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = albert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(20,activation = 'tanh')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [92]:
optimizer = Adam(
    learning_rate=5e-05, 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = MeanSquaredError()
metric = MeanSquaredError()
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = [MeanSquaredError(),MeanAbsolutePercentageError()])

In [93]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=36
)

 200/1112 [====>.........................] - ETA: 2:04:27 - loss: 1.3386 - mean_squared_error: 1.3386 - mean_absolute_percentage_error: 2510906.2500

KeyboardInterrupt: 

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

32/32 [==============================] - 24s 760ms/step


array([ 0.11043661,  0.250574  ,  0.3487891 ,  0.28715646,  0.03419731,
        0.17019227, -0.03131878,  0.44810688,  0.03534355,  0.02628882,
        0.22291204,  0.20075162,  0.3036334 ,  0.17166708,  0.26290175,
        0.29933837,  0.2107129 ,  0.00052988,  0.29390728, -0.03019417],
      dtype=float32)

In [ ]:
plt.plot(train_history.history['balanced_accuracy'])
plt.plot(train_history.history['val_balanced_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

KeyError: 'balanced_accuracy'

In [ ]:
user_id = test_df['user_id'].tolist()[0]
print(user_id)

1931


In [ ]:
newVector = np.array(predicted_raw[0])
print(newVector)
modelRec.user_factors[user_id]

[ 0.11043661  0.250574    0.3487891   0.28715646  0.03419731  0.17019227
 -0.03131878  0.44810688  0.03534355  0.02628882  0.22291204  0.20075162
  0.3036334   0.17166708  0.26290175  0.29933837  0.2107129   0.00052988
  0.29390728 -0.03019417]


array([-0.16504681, -1.299497  , -0.73947155, -0.27395138,  0.54700875,
        0.9955758 ,  0.67054194,  2.2759151 , -0.13877383, -0.01356953,
       -1.4546584 , -1.1597674 , -1.01425   , -0.87998253,  2.3494205 ,
        1.9512452 ,  1.4806257 ,  0.20059037, -0.5299491 ,  1.3627071 ],
      dtype=float32)

In [ ]:
modelRec.recommend(user_id,data_conf[user_id], N = 20)

(array([1351, 2442,  652, 1716,  250, 1093, 2239, 1538, 2210, 1013]),
 array([0.6310415 , 0.56917197, 0.56360924, 0.5405733 , 0.50737274,
        0.47627532, 0.4468179 , 0.44079867, 0.43895715, 0.419195  ],
       dtype=float32))

In [ ]:
modelRec.user_factors[user_id] = newVector
print(modelRec.recommend(user_id,data_conf[user_id], N =20))

(array([2166,  469, 1342,  872, 2613,  252, 2478,  171, 2084, 1810,  666,
        518,  134, 1964,  756,  423,  465,  440,  630,  195, 1827, 2349,
       1185, 2261, 1686,  555, 2491,  107, 1676,   40]), array([0.10468029, 0.09051982, 0.08740012, 0.08347838, 0.08332894,
       0.08098835, 0.08068367, 0.07956488, 0.0774186 , 0.07591844,
       0.07547419, 0.07518977, 0.07424901, 0.0727215 , 0.07023704,
       0.06971195, 0.06905746, 0.06793718, 0.06787977, 0.0671107 ,
       0.06567258, 0.06548437, 0.06544199, 0.06511699, 0.06510273,
       0.06482379, 0.06472807, 0.06373616, 0.0632884 , 0.06257409],
      dtype=float32))
